<a href="https://colab.research.google.com/github/VMutafchiev/LLMs/blob/main/4_0_5_TLC_FAQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain==0.0.228 --progress-bar off
!pip install -qqq chromadb==0.3.26 --progress-bar off
!pip install -qqq sentence-transformers==2.2.2 --progress-bar off
!pip install -qqq auto-gptq==0.2.2 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq unstructured==0.8.0 --progress-bar off
!pip install -qqq transformers==4.30.2 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off

!pip install google-cloud-aiplatform==1.25.0 --progress-bar off
!pip install langchain==0.0.187 --progress-bar off
!pip install xmltodict==0.13.0 --progress-bar off
!pip install unstructured==0.7.0 --progress-bar off # used by langchain
!pip install pdf2image==1.16.3 --progress-bar off #used by langchain
!pip install requests==2.31.0 --progress-bar off
!pip install beautifulsoup4==4.12.2 --progress-bar off



  Obtaining dependency information for langchain==0.0.187 from https://files.pythonhosted.org/packages/53/3e/ab13413ffecea43820a573b59f313bc55ef3f80c1d98c72f92de36d881f3/langchain-0.0.187-py3-none-any.whl.metadata
  Using cached langchain-0.0.187-py3-none-any.whl.metadata (12 kB)
Using cached langchain-0.0.187-py3-none-any.whl (960 kB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.228
    Uninstalling langchain-0.0.228:
      Successfully uninstalled langchain-0.0.228
  Obtaining dependency information for unstructured==0.7.0 from https://files.pythonhosted.org/packages/ae/f7/da376de8562a7995d3a1a4127199e03295b7decc5605b364574fa1ede5ed/unstructured-0.7.0-py3-none-any.whl.metadata
  Using cached unstructured-0.7.0-py3-none-any.whl.metadata (16 kB)
Using cached unstructured-0.7.0-py3-none-any.whl (1.3 MB)
  Attempting uninstall: unstructured
    Found existing installation: unstructured 0.8.0
    Uninstalling unstructured-0.8.0:
      Successfully unins

In [ ]:
from pathlib import Path

import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, GenerationConfig, TextStreamer, pipeline

from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import argparse

from sentence_transformers import SentenceTransformer

import warnings

In [ ]:
import requests
from bs4 import BeautifulSoup

def parse_sitemap(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "xml")
    texts = [element.text for element in soup.find_all("p")]
    return texts
sitemap='https://tlc.thinkorswim.com/center'
sites = parse_sitemap(sitemap)

In [ ]:
import re
import os

def sanitize_file_name(name):
            # Remove any characters that are not allowed in file names
            return re.sub(r'[\\/:"*?<>|]', '', name)

def scrape_and_save_urls(urls, output_directory):
    for index, url in enumerate(urls, start=1):
        try:
            response = requests.get(url)
            response.raise_for_status()

            content = response.content.decode("utf-8")
            soup = BeautifulSoup(content, "html.parser")

            #Extract FAQ into separate pages
            faq_entries = soup.find_all('article')

            for index, entry in enumerate(faq_entries, start=1):
              questionFAQ = entry.find('h2').text.strip()
              answerFAQ = entry.find('p').text.strip()

              sanitized_question = sanitize_file_name(questionFAQ)

            # Create a file name based on the question and save the question and answer to a separate file
              file_name = f"{sanitized_question}.txt"
              file_path = os.path.join(output_directory, file_name)

              with open(file_path, "w", encoding="utf-8") as f:
                f.write(f"Question: {questionFAQ}\n\n")
                f.write(f"Answer: {answerFAQ}\n")



        except requests.exceptions.RequestException as e:
            print(f"Error fetching the URL {url}: {e}")
        except Exception as e:
            print(f"Error processing the URL {url}: {e}")

# Example usage:
manual_urls = [
    "https://tlc.thinkorswim.com/center/faq/general",
    # Add more URLs manually as needed
]

output_directory = 'FAQ'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

scrape_and_save_urls(manual_urls, output_directory)


In [ ]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda:0'

In [ ]:
model_name_or_path = "TheBloke/Nous-Hermes-13B-GPTQ"
model_basename = "nous-hermes-13b-GPTQ-4bit-128g.no-act.order"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device=DEVICE,
        use_triton=use_triton,
        quantize_config=None)

In [ ]:
generation_config = GenerationConfig.from_pretrained(model_name_or_path)
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.30.2"
}

In [ ]:
question = (
    "Which is Level 2?"
)
prompt = f"""
### Instructions: {question}
### Response:
""".strip()

In [ ]:
%%time
input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(DEVICE)
with torch.inference_mode():
  output = model.generate(
      inputs=input_ids,
      temperature=0.5,
      max_new_tokens=512)

CPU times: user 5.17 s, sys: 494 ms, total: 5.67 s
Wall time: 8.28 s


In [ ]:
streamer = TextStreamer(
    tokenizer,
    skip_prompt=True,
    skip_special_tokens=True,
    use_multiprocessing=False
)

In [ ]:
# Inference can also be done using transformers' pipeline
# Prevent printing spurious transformers error when using pipeline with AutoGPTQ
logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=4048,
    temperature=0,
    top_p=0.9,
    repetition_penalty=1.2,
    generation_config=generation_config,
    streamer=streamer,
    batch_size=1,
)

In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)
responce = llm(prompt)

The margin of a page refers to the blank space that surrounds the text and other elements on a printed page. It is usually measured in millimeters or inches, with common sizes being 1", 1.25" or 1.5". The purpose of having margins is to provide breathing room for the reader's eyes and to make it easier to hold and turn the pages of a book or document.


In [ ]:
model = SentenceTransformer("embaas/sentence-transformers-multilingual-e5-base")

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name='embaas/sentence-transformers-multilingual-e5-base',
    model_kwargs={"device":DEVICE}
)

In [ ]:
questions_dir = Path("FAQ")

In [ ]:
DEFAULT_TEMPLATE = """
### Instructions: Follow exactly those 3 steps:
1. Read the context below and aggregrate this data
Context{context}.
2. Answer the question using only this context.
Answer in a helpful and polite manner to the question. If you don't know the answer - make an excuse and say that you don't know.
Keep your replies short, compassionate and informative.
{chat_history}
### Input: {question}
### Response:
""".strip()


class Chatbot:
    def __init__(
        self,
        text_pipeline: HuggingFacePipeline,
        embeddings: HuggingFaceEmbeddings,
        documents_dir: Path,
        prompt_template: str = DEFAULT_TEMPLATE,
        verbose: bool = False,
    ):
        prompt = PromptTemplate(
            input_variables=["context", "question", "chat_history"],
            template = prompt_template,
        )
        self.chain = self._create_chain(text_pipeline, prompt, verbose)
        self.db = self._embed_data(documents_dir, embeddings)

    def _create_chain(
        self,
        text_pipeline: HuggingFacePipeline,
        prompt: PromptTemplate,
        verbose: bool = False,
    ):
        memory = ConversationBufferMemory(
            memory_key = "chat_history",
            human_prefix = "### Input",
            ai_prefix = "### Response",
            input_key = "question",
            output_key = "output_text",
            return_messages=False,
        )
        return load_qa_chain(
            text_pipeline,
            chain_type="stuff",
            prompt=prompt,
            memory=memory,
            verbose=verbose,
        )

    def _embed_data(
        self, documents_dir: Path, embeddings: HuggingFaceEmbeddings
      ) -> Chroma:
          loader = DirectoryLoader(documents_dir, glob="**/*txt")
          documents = loader.load()
          text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
          texts = text_splitter.split_documents(documents)
          return Chroma.from_documents(texts, embeddings)

    def __call__(self, user_input: str) -> str:
        docs = self.db.similarity_search(user_input)
        return self.chain.run({"input_documents": docs, "question": user_input})

In [ ]:
chatbot = Chatbot(llm, embeddings, "./FAQ/")

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning)

while True:
    user_input = input("Question: ")
    if user_input.lower() in ["bye", "goodbye", "see ya", "thank you bye"]:
        break
    answer = chatbot(user_input)
    print()


Question: What is level 2?
Level 2 refers to the second layer of data available when analyzing financial instruments such as stocks or options. It provides additional details about orders placed by market participants, including their size and price. This information can be useful for traders who want to understand more about what is driving the current price movement of a particular security.

Question: I have a problem I want to contact support. 
Of course! Please provide me with some more details so I can assist you better.

Question: I need a phone number
Sure thing! The customer service phone number for TD Ameritrade is (800) 457-3479.

Question: how to fund my account?
Hi there! There are several ways to fund your account with TD Ameritrade. One option is to use electronic ACH which allows you to directly transfer funds from your bank account to your brokerage account. Another option is to use wire transfers where you can send money electronically from another institution. For mo